# Reproduction of "Driver Identiﬁcation Based on Vehicle Telematics Data using LSTM-Recurrent Neural Network" using pytorch

Author: Lorenzo Pes (5604958)

The main objective of this jupyter notebook is to provide a replication of the paper [Driver Identiﬁcation Based on Vehicle Telematics Data using LSTM-Recurrent Neural Network](https://arxiv.org/abs/1911.08030) using PYtorch to create the neural model proposed. The main objective of the paper is to train a model which can identify the driver of a car based on his driving behaviour. This allow the model to identify in real-time if the driver of the car is the owner or someone else. To achieve this goal the paper attempts to train a Recurrent Neural Network (RNN) with LSTM cells, using sensors data obtained from OBD-II. The dataset is comprised of measurements such as vehicle speed, throttle position, break pedal displacement etc. Specifically, this notebooks attempts to reproduce the LSTM results of figure 9 and 10 of the paper shown below. 

![alt text](f9.png "Figure 9") 
<center>Figure 9 </center>

In the Figure 9, the authors of the paper investigated how different levels of white gaussian noise in the input sensors data would affect the accuracy of a trained model. In addition to that, the autorhs compare the results of their LSTM model with other algorithms such as FCNN, Decision Tree and Random Forest demostrating that the LSTM network is the most insensitive to noisy sensor data. 

![alt text](f10.png "Figure 10") 
<center>Figure 10 </center>

On the other end, in Figure 10, the authors demonstrate the accuracy on the test set for the same algorithms as before, but with the model trained with noisy data. Even for this set-up the LSTM shows the better accuracy against its competitors. 

## The Dataset

The authors test their model with two different datasets: Security Driving Dataset and Vehicular data trace Dataset-1. Unfortunatelly, due to these datasets not being publicly available, another similar [dataset](https://ocslab.hksecurity.net/Datasets/driving-dataset) was used. It consists of recorded sensor data from 10 different drivers of a trip of about 46km, between Korea University and SANGAM World Cup Stadium. Let's start investigating the dataset with pandas. 

### Inport the functions required 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

I am now going to load the dataset and display some useful informations. Lets first display the name of all the columns of the dataset 

### Features name of the dataset 

In [ ]:
df = pd.read_csv('dataset.csv')
for col_name in df.columns: 
    print(col_name)

### Sensor recordings per driver class 
I will not plot the number of recordings for each driver in a histogram fromat. 

In [ ]:
df['Class'].value_counts().sort_index().plot(kind='bar', title='Recordings vs Driver')
plt.ylabel('Recordings')
plt.xlabel('Driver')

The Figure above shows the number of recordings for each driver. It can be noticed that in this dataset, the number of recorded sensor data is not equivalent for each driver. For instance, drive B and D posses one order of magnitude more data that the avarage of the other drivers. In the next cells I will print the histogram for each column of the dataset.

### Histogram of dataset

In [ ]:
hist = df.hist(bins=100,figsize=(36,20))
plt.show()


From the figure above, it can be seen that some recording are constantly zero. For istance the measurements of **Filtered_Accelerator_Pedal_value, Inhibition_of_engine_fuel_cut_off, Fuel_Pressure, Torque_scaling_factor(standardization) and Glow_plug_control_request** are constantly zero. Consequently, these values can be dropped out from the dataset when training the network because input features with all zeors will be insignificant for the training of the network.

## Data pre-processing

In this section of the notebook, it will be explained how I have pre-processed the dataset to feed in the network. Clearly, this step is based on the description provided in the paper. However, some ambiguities were found in the author description and they will be further investigated in the next sections. To begin with, the range of sensors recordings are not equal (i.e **engine_cooling_temperature** measurements are between 75 and 100, while **torque_converter_speed** measurements are between 0 and 3000). This umbalance of ranges will make the **torque_converter_speed** recordings more influential than **engine_cooling_temperature** due to its larger range. Furthermore, the authors frame the problem as a time-series prediction. That is, a temporal series of sensors features is fed to the input of Recurrent Neural Networks and predicts the deriver of that sequence. To achieve this goal, the dataset must be split into multiple windows as explained further in the notebook.

### Normalization Functions
For the training of the dataset, we would like all the sensors readings to have the same influence over the network. To achieve the former, the authors states in the paper that the dataset is standardized using min-max normalization. Unfortunately, this is not proven to be the same in the code provided by the authors. For instance, by looking at the function $normalize()$ in [provided code](https://github.com/Abeni18/Deep-LSTM-for-Driver-Identification-/blob/master/main.py) it seems that the data is actually normalized using a $StandardScaler()$ rather than a min-max standardization. Consequently, in this reproduction both normalization techniques are implemented to study how the affect the resulting accuracy of the network.

#### Min-Max Normalizer

In [ ]:
def min_max_norm(self,col):
    self._data[col]=(self._data[col]-self._data[col].min())/(self._data[col].max()-self._data[col].min())

#### Standard Scaler

In [ ]:
def std_scaler(self,col):
    self._data[col]=(self._data[col]-self._data[col].mean())/(self._data[col].std())

### Windowing and Pytorch Dataloader
As previously mentioned, the input of the recurrent network is a sequence of time instances containing all the recorded features. Figure 5 taken from the paper gives and example of the former.

![alt text](window.png "Figure 5") 
<center>Figure 5 </center>

It can be seen that the the windows are overlapping by 50% to smooth the flow of sequence. Furthermore, the authors state that after hyperparameter tuning it was found that the optimal windows size consists of 16 consecutive time instances. Since the number of measurements for each class is not the same and the number of recordings for a given class might not be exactly divisible by the window size, some recording were dropped to avoid that a temporal sequence might contains values from another driver (class). Sadly, the authors do not elaborate in this regard and therefore this intuition might not be exatly what is done in the implementation, but it is still a safe assumption to make. Perhaps, by looking at the code implementation, it seems that the authors do not perform this dropping but rather choose the most occuring driver for a griven temporal sequence. However, the results should not be affected by this difference in framing the dataset. Also, the output class must be encoded into a numerical values to be understood by the network. Both functions have been implemented inside the constructor of a pytorch dataloader as shown below. 

In [ ]:
import torch
import numpy as np
from sklearn import preprocessing
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, file_path='dataset.csv', classes_to_drop=['Class', 
                                                                 'PathOrder', 
                                                                 'Time(s)', 
                                                                 'Filtered_Accelerator_Pedal_value',
                                                                 'Inhibition_of_engine_fuel_cut_off',
                                                                 'Fuel_Pressure',
                                                                 'Torque_scaling_factor(standardization)',
                                                                 'Glow_plug_control_request'], window_size=16, normalize=True, normalize_method='mean_std'):
        
        
        self._window_size=window_size
        self._data=pd.read_csv(file_path)
        
        # The data is sorted by Class A,B,C the indexes of the dataframe have restarted by ignore index
        self._data = self._data.sort_values(by=['Class'], inplace=False,ignore_index = True) 
        
        # class_uniq contains the letters of the drivers A,B and it loops across all of them 
        for class_uniq in list(self._data['Class'].unique()):
            # find the total number of elements belonging to a class
            tot_number=sum(self._data['Class']==class_uniq) 
            #number of elements to drop so that the class element is divisible by window size 
            to_drop=tot_number%window_size
            #returns the index of the first element of the class
            index_to_start_removing=self._data[self._data['Class']==class_uniq].index[0]
            #drop element from first element to the element required 
            self._data.drop(self._data.index[index_to_start_removing:index_to_start_removing+to_drop],inplace=True)
            
        
        #resetting index of dataframe after dropping values 
        self._data = self._data.reset_index()
        self._data = self._data.drop(['index'], axis=1)
        
        index_starting_class=[] # This array contains the starting index of each class in the df
        for class_uniq in list(self._data['Class'].unique()):
            #Appending the index of first element of each clas
            index_starting_class.append(self._data[self._data['Class']==class_uniq].index[0])
        
        # Create the sequence of indexs of the windows 
        sequences=[]
        for i in range(len(index_starting_class)):
            #check if beginning of next class is there 
            if i!=len(index_starting_class)-1:
                ranges=np.arange(index_starting_class[i], index_starting_class[i+1])       
            for j in range(0,len(ranges),int(self._window_size/2)):
                if len(ranges[j:j+self._window_size])==16:
                    sequences.append(ranges[j:j+self._window_size])
        self._sequences=sequences
    
        
        #take only the 'Class' which are the actual labels and store it in the labels of self 
        self._labels=self._data['Class']
        # Dropping columns which have constant measurements because they would return nan in std
        self._data.drop(classes_to_drop, inplace=True, axis=1)
        
        # Function to normalize the data either with min_max or mean_std
        if normalize:
            for col in self._data.columns:
                if normalize_method=='min_max':
                    min_max_norm(self,col)
                elif normalize_method=="mean_std":
                    std_scaler(self,col)
                    
        # Create the array holding the windowed multidimensional arrays 
        X=np.empty((len(sequences), self._window_size, len(self._data.columns)))
        y=[]
        
        for n_row, sequence in enumerate(sequences):
            X[n_row,:,:]=self._data.iloc[sequence]
            # I am saying that the corresponding driver of the sequence is the driver at first sequence
            # This should be OK but not sure, maybe take the most recurrent driver in the sequence ? 
            y.append(self._labels[sequence[0]]) 
            #y.append(self._labels.iloc[sequence])
       

        assert len(y)==len(X)
        #Assing the windowed dataset to the X of self 
        self._X= X
        
        # targets is a transformed version of y with drivers are encoded into 0 to 9 
        targets = preprocessing.LabelEncoder().fit_transform(y)
        targets = torch.as_tensor(targets) # just converting it to a pytorch tensor
        self._y=targets # assign it to y of self 
        
        #import pdb; pdb.set_trace()

        
    def __len__(self):
        return len(self._X)
        
    
    def __getitem__(self, index):
        return torch.FloatTensor(self._X[index,:,:]), self._y[index]


At this point we have a fully customized dataloader for pytorch which will allow us to create the dataloaders for the traning, validation and test sets. Lets start first my creating an instance of the custom dataloader as shown below. 

In [ ]:
a = CustomDataset()

### Train, Validation and Test set

The authors of the papers splits the dataset in the following way:

* 85% training  
* 5%  validation
* 10% test 

The same will be done in our replication as shown in figure below. Also, the optimal batch size found by the authors will be initialized. 

In [ ]:
train_size = int(0.85 * len(a))
val_size = int(0.05 * len(a))
test_size = len(a)-train_size-val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(a, [train_size,val_size, test_size])

batch_size = 32

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           drop_last=True)
 
validation_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False,
                                           drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Recurrent Neural Network
In this section of the notebook the implmentation of the Recurrent Neural Network with LSTM cells will be explained. To begin with, the paper implements a many-to-one RNN two hidden layers composed of LSTM cells. Figure 3 taken form the paper, provides a visual representation of the network. 

![alt text](net.png "Figure 3") 
<center>Figure 3 </center>

As it can be seen in the image, a temporal sequence and all the relative sensors data are fed into the first hidden layer, above which, a second hidden layer is stacked. In the papers it is stated that "the last hiddel layer feature vector is taken and used to output a classification score for the given input sequence". Understanding this part is of fundamental importance for the correct reproduction of the paper. Nevertheless, the authors spend no more that one paragraph in this topic and provide an image that does not really clarify how the output classification is performed. Furthermore, the authors state that "the last layer of the is a sigmoid function". It is really unclear what is meant by the former sentence because the sigmoid function is just an activation function of a given layer and not the layer itself. Perhaps, the last layer is actually a fully connected layer with an output dimension of 10, one for each class. In the replication of the code, the last layer is considered as as fully connected linear layer with a sigmoid activation function at its output. 

Another important aspect to be taken into account for the declaration of the model is the number of neurons per hidden layer. After hyper-parameter tuning, the authors found the best number of neurons to be **160 for the first hidden** layer and **200 for the second hidden layer**. Furthermore, to ease the training process batch normalization has been indroduced at the output of each LSTM hidden layer. The class implementing the network is shown in the cell below. 

### Neural Network Model

In [ ]:
class NEURAL(torch.nn.Module):
    
    def __init__(self, batch_size, window_size, num_features):
        super(NEURAL, self).__init__()
        self.lstm1 = torch.nn.LSTM(num_features, 160,batch_first = True)
        #self.bn1 = torch.nn.BatchNorm1d(16)
        self.lstm2 = torch.nn.LSTM(160, 200, 1)
        #self.bn2 = torch.nn.BatchNorm1d(16)     
        self.fc = torch.nn.Linear(200, 10)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        lstm1_out, (h_t1, c_t1) = self.lstm1(x)  
        #self.bn1(lstm1_out)
        lstm2_out, (h_t2, c_t2) = self.lstm2(lstm1_out)  
        #self.bn2(lstm2_out)
        fc_out = self.fc(lstm2_out)
        out = self.sigmoid(fc_out)
        
        return out[:,-1,:]


Now lets create an instance of the network

In [ ]:
inputs, classes = next(iter(train_loader)) 
model = NEURAL(inputs.shape[0],inputs.shape[1],inputs.shape[2])

### Training loop

As specified by the paper, the loss function used to train the network is the normal Cross Entropy Loss. On the other end, the authors  do not specify the learning rate nor the number of epochs nor the optimizer. In the reproduction of the paper the learning rate has been initialized with a value of 0.001, but set to decay exponentially with the number of epochs as defined by the function *lambda1*. Furthermore, Adam optimizer was choosen even if better results on the test set might be achivable with SGD. Also, for the number of epochs, different test were performed to find a number of epochs equal to x that leads to an F1 score similar to the one provided by the authors. 

 

In [ ]:
learning_rate = 1e-3
num_epochs = 50

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lambda1 = lambda epoch: 0.99 ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

def train_one_epoch(epoch):
    running_loss = 0
    print("Epoch:", epoch)
    for i, (data, labels) in enumerate(train_loader):
        
        optimizer.zero_grad()
        # Make predictions for this batch
        outputs = model(data)
        # Compute the loss and its gradients
        loss = criterion(outputs,labels)
        loss.backward()
        # Compute the loss and its gradients
        optimizer.step()
        running_loss += loss.item()
       
    
    print('Loss Avg: {:.6f}'.format(running_loss/len(train_loader)))
            
    return running_loss/len(train_loader)

epoch_number = 0
train_loss = []
for epoch in range(num_epochs):

    model.train(True)
    train_loss.append(train_one_epoch(epoch))
    scheduler.step()
    epoch_number += 1

In the next cell I will just save the model 

In [ ]:
torch.save(model.state_dict(), 'noiseless_model/model.pt')

### Average Loss vs Epochs Plot

In [ ]:
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(train_loss)
plt.grid()

### Metrics 
The paper uses F1-score to measure the performance of the model in the test set. This metric is defined by the following set of equations:

$$ F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall}  $$

$$ Precision = \frac{TP}{TP+FP}  $$

$$ Recall = \frac{TP}{TP+TN}  $$

In the equations above the variables have the following meanings: 

* TP: True Positive 
* FP: False Positive 
* FN: False Negative 

The F1 scores can be computed for each class of the drivers. From the papers it can be deduced that the total F1 score used to measure the accuracy of the model is the average of each class. Fortunatelly, scikit-learn provides a function to calculate the F1 score for each class given a vector of true and predicted labels. The function to calculate the accuracy of the model is shown in the cell below. 

In [ ]:
from sklearn.metrics import f1_score

inputs, classes = next(iter(train_loader)) 
model_noiseless = NEURAL(inputs.shape[0],inputs.shape[1],inputs.shape[2])
model_noiseless.load_state_dict(torch.load('noiseless_model/model.pt'))
model_noiseless.eval()



def f1(test_loader,model):
    f1 = 0
    with torch.no_grad():
        for i, (data, labels) in enumerate(test_loader):
            outputs = model(data)
            pred = outputs.data.max(1, keepdim=True)[1] 
            f1 += f1_score(labels, pred, average='macro')
        
    avg_f1 = f1/len(test_loader) 
    
    return(avg_f1*100)

f1(test_loader,model_noiseless)

##  Noise 

In this part of the notebook the effect of adding noise with different standard deviations to the sensor readings will be investigated. Frist, the accuracy of the model trained with noiseless data will be calculated for different level of noise added to the input vectors. After, a new model with noisy input data will be trained to calculate how much the accuracy degrades and if there is improvements. 

The authors are slightly unclear on their definitions of noise, I am implementing the noise as a random signal with 0 mean and a selectable standard deviation as shown below. 

In [ ]:
def add_noise(a,std):
    noise = np.random.normal(0,std,a._X.shape)
    a._X = a._X + noise 


In [ ]:
accuracies = []
std_levels = [0,0.25,0.5,0.75,1,1.25,1.5,1.75,2]

for std in std_levels:
    dataset = CustomDataset()
    add_noise(dataset,std)
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size,val_size, test_size])
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)
    accuracies.append(f1(test_loader,model_noiseless))

In [ ]:
fig, ax = plt.subplots()
plt.plot(np.arange(0,2.25, 0.25),accuracies)
ax.set_ylim(ymin=0)
plt.xlabel("Noise STD")
plt.ylabel("Accuracy")
plt.title("LSTM - Accuracy vs Noise STD")
plt.grid()



In [ ]:
inputs, classes = next(iter(train_loader))

out = model(inputs)
print(out)
for element in out:
    print(torch.argmax(element))
print(classes)
